# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [4]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [5]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [6]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [7]:
# Import the CSVs to Pandas DataFrames
file_path = Path("/Users/simar/Jupyter-Workspace/4. Data Visualization/PyViz Homework/Resources/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("/Users/simar/Jupyter-Workspace/4. Data Visualization/PyViz Homework/Resources/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Panel Visualization Functions

In [13]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    to_new_data = to_data.groupby('neighbourhood').mean()
    to_data_map = to_new_data.reset_index()
    avg_value_location = pd.concat([to_data_map, df_neighbourhood_locations], axis="columns", join="inner")
    
    
    to_values = px.scatter_mapbox(
        avg_value_location,
        lat="lat",
        lon="lon", 
         
        hover_data=["average_house_value","single_detached_house","apartment_five_storeys_plus","movable_dwelling","semi_detached_house","row_house","duplex","apartment_five_storeys_less","other_house","shelter_costs_owned","shelter_costs_rented"],
        color ="average_house_value",
        size = "average_house_value",
        title = "Average House Values in Toronto",
        color_continuous_scale=px.colors.sequential.Rainbow, 
        size_max=10,
       # hover_name = "neighbourhood",
    )
    return to_values
    
def create_bar_chart():
    """
    Create a barplot based in the data argument.
    """
    
    to_dwelling_data = to_data.groupby('year').sum()
    to_dwelling_data = to_dwelling_data.drop(['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'], axis=1)
    
    to_data_2001 = to_dwelling_data.iloc[0]
    plot_to_data_2001 = to_data_2001.hvplot.bar(xlabel= "2001" , ylabel =" Dwelling Type Units" ,label='Dwelling Types in Toronto in 2001' , rot =90).opts(
    yformatter="%.0f" , width = 400 , height =500 , color = "red" , line_color = "white")



    to_data_2006 = to_dwelling_data.iloc[1]
    plot_to_data_2006 = to_data_2006.hvplot.bar(xlabel= "2006" , ylabel =" Dwelling Type Units" ,label='Dwelling Types in Toronto in 2006' , rot =90).opts(
    yformatter="%.0f" , width = 400 , height =500 , color = "blue" , line_color = "white")



    to_data_2011 = to_dwelling_data.iloc[2]
    plot_to_data_2011 = to_data_2011.hvplot.bar(xlabel= "2011" , ylabel =" Dwelling Type Units" ,label='Dwelling Types in Toronto in 2011' , rot =90).opts(
    yformatter="%.0f" , width = 400 , height =500 , color = "orange" , line_color = "white")



    to_data_2016 = to_dwelling_data.iloc[3]
    plot_to_data_2016 = to_data_2016.hvplot.bar(xlabel= "2016" , ylabel =" Dwelling Type Units" ,label='Dwelling Types in Toronto in 2016' , rot =90).opts(
    yformatter="%.0f" , width = 400 , height =500 , color = "magenta" , line_color = "white")
    
    return plot_to_data_2001+plot_to_data_2006+plot_to_data_2011+plot_to_data_2016
     


def create_line_chart():
    """
    Create a line chart based in the data argument.
    """
    
    to_shelter_data = to_data.groupby('year').mean()
    to_shelter_data = to_shelter_data.drop(['average_house_value', 'single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house' ], axis=1)
    
    owned_shelter = to_shelter_data['shelter_costs_owned']
    plot_owned_shelter = owned_shelter.hvplot(xlabel= "Year" , ylabel =" Avg. Monthly Shelter Cost",label='Average Monthly Shelter Cost for Owned Dwellings in Toronto',rot =90 , color ="blue" , width =600)

    rented_shelter = to_shelter_data['shelter_costs_rented']
    plot_rented_shelter = rented_shelter.hvplot(xlabel= "Year" , ylabel =" Avg. Monthly Shelter Cost" ,label='Average Monthly Shelter Cost for Rented Dwellings in Toronto',rot =90 , color ="orange" , width =600)

    return plot_owned_shelter + plot_rented_shelter
     


def average_house_value():
    """Average house values per year."""
    
    avg_house_value = to_data["average_house_value"].groupby([to_data.index]).mean()
    plot_avg_house_value= avg_house_value.hvplot(
        xlabel= "Year",
        ylabel =" Average House Value", 
        label='Average House Value in Toronto' , 
        rot =90 , 
        color ="purple" , 
        width =600).opts(
        yformatter="%.0f")
    
    return plot_avg_house_value

    
def average_value_by_neighbourhood(neighbourhood):
    """Average house values by neighbourhood."""
    
    to_data_new = to_data.groupby([to_data.index, "neighbourhood"]).mean()
    to_house_value = to_data_new["average_house_value"]
    to_house_value = pd.DataFrame(to_house_value).reset_index()

    return to_house_value.loc[to_house_value['neighbourhood']==neighbourhood,:].hvplot.line(
        x="year",
        y="average_house_value",
        xlabel = "Year",
        ylabel = " Average House Value",

        title="Average House Value by Neighbourhood", yformatter="%.0f"
    )
to_data_new = to_data.groupby([to_data.index, "neighbourhood"]).mean()
to_house_value = to_data_new["average_house_value"]
to_house_value = pd.DataFrame(to_house_value).reset_index()
neighbourhood_choice = to_house_value["neighbourhood"]
interact(average_value_by_neighbourhood, neighbourhood=neighbourhood_choice)


def number_dwelling_types(neighbourhood):
    """Number of dwelling types per year"""
    
    dwelling_data = to_data.drop(["average_house_value","shelter_costs_owned","shelter_costs_rented"] , axis = "columns")
    return dwelling_data.loc[dwelling_data['neighbourhood']==neighbourhood,:].hvplot.bar(
        xlabel = "Year",
        ylabel = "Dwelling Type Units",
        title="Number of Dwelling Types per Year by Neighbourhood", yformatter="%.0f", rot =90, width = 900 , height = 400
    )

dwelling_data = to_data.drop(["average_house_value","shelter_costs_owned","shelter_costs_rented"] , axis = "columns")
neighbourhood_choice = dwelling_data["neighbourhood"]
interact(number_dwelling_types, neighbourhood=neighbourhood_choice)


def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    to_data_new = to_data.groupby([to_data.index, "neighbourhood"]).mean()
    to_house_value = to_data_new["average_house_value"]
    to_house_value = pd.DataFrame(to_house_value).reset_index()
    
    snapshot = px.bar(
        to_house_value, 
        x = "neighbourhood",
        y = "average_house_value",
        color = "average_house_value", 
        height = 1000, 
        facet_row ="year")
    return snapshot


def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    top_10_data = to_data.groupby('neighbourhood').mean().drop(['shelter_costs_owned', 'shelter_costs_rented'], axis=1).sort_values(by ='average_house_value' ,ascending = False)
    top_10_values = top_10_data.nlargest(10, 'average_house_value')
    
    plot_top_10_values = top_10_values.hvplot.bar(
        y = "average_house_value", 
        xlabel= "Neighbourhood", 
        ylabel ="Average House Value",
        label='Top 10 Expensive Neighbourhoods in Toronto', 
        rot =90).opts(
        yformatter="%.0f" , 
        width = 700 , 
        height =500 , 
        color = "blue" )
    
    return plot_top_10_values

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [16]:
# Create a Title for the Dashboard
title = '##Real Estate Analysis of Toronto from 2001-2016'

# Define a welcome text
welcome_text = ("This Dashboard represents the visual analysis of historical house values, dwelling types per neighbourhood and dwelling costs in Toronto, Ontario according to census data from 2001- 2016. You can navigate through the tabs above to explore more details on the evolution of real estate market on The 6 across these years.")

# Create a tab layout for the dashboard
welcome_column = pn.Column(title , welcome_text, neighbourhood_map())

yearly_analysis_column = pn.Row(create_bar_chart())

shelter_vs_value_column = pn.Column(create_line_chart(), average_house_value())

neighbourhood_column = pn.Column(interact(average_value_by_neighbourhood, neighbourhood=neighbourhood_choice), interact(number_dwelling_types, neighbourhood=neighbourhood_choice),
average_house_value_snapshot())

expensive_column = pn.Column(top_most_expensive_neighbourhoods())


# Create the main dashboard
dashboard = pn.Tabs(
    ("Welcome", welcome_column), 
    ("Yearly Market Analysis", yearly_analysis_column), 
    ("Shelter Costs vs House Values" , shelter_vs_value_column), 
    ("Neighbourhood Analysis", neighbourhood_column), 
    ("Top Expensive Neighbourhoods", expensive_column) 
)

## Serve the Panel Dashboard

In [17]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Plotly(Figure)
    [1] Row
        [0] HoloViews(Layout)
    [2] Column
        [0] HoloViews(Layout)
        [1] HoloViews(Curve)
    [3] Column
        [0] Column
            [0] Column
                [0] Select(name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North')
            [1] Row
                [0] HoloViews(Curve, name='interactive07598')
        [1] Column
            [0] Column
                [0] Select(name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North')
            [1] Row
                [0] HoloViews(Bars, name='interactive07683')
        [2] Plotly(Figure)
    [4] Column
        [0] HoloViews(Bars)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [24]:
#neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
#create_bar_chart(to_data_2001.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
#average_house_value()

In [ ]:
#average_value_by_neighbourhood()

In [ ]:
#number_dwelling_types()

In [ ]:
#average_house_value_snapshot()

In [ ]:
#top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()